------------------------------- MACRO DATA

In [21]:
# Preprocess data

import numpy as np
import pandas as pd

In [22]:
# Filepaths

sp500_data = 'MacroData/sp500_data.csv'
bond_data = 'MacroData/10_year_bonds.csv'
inflation_data = 'MacroData/inflation_data.csv'

difficulty_data = 'BitcoinData/bitcoin_network_difficulty.csv'
hashrate_data = 'BitcoinData/bitcoin_network_hashrate.csv'
btc_prices = 'BitcoinData/bitcoin_prices.csv'

In [23]:
def rename_cols(df, cols, names):
    new_names = dict(zip(cols, names))
    df.rename(columns=new_names, inplace=True)
    return df

def fill_empty_dates(df, date_col):
    date_range = pd.date_range(start=df[date_col].min(), end=df[date_col].max())
    full_dates_df = pd.DataFrame(date_range, columns=['Date'])
    full_dates_df['Date'] = pd.to_datetime(full_dates_df['Date'])
    df = pd.merge(full_dates_df, df, on='Date', how='left')
    df.ffill(inplace=True)
    return df

In [24]:
sp500 = pd.read_csv(sp500_data)

sp500['Date'] = pd.to_datetime(sp500['Date'])
sp500 = sp500.drop(columns='Volume')
sp500 = rename_cols(sp500, ['Open', 'Close', 'High', 'Low'], ['sp500_open', 'sp500_close', 'sp500_high', 'sp500_low'])
sp500 = fill_empty_dates(sp500, 'Date')

sp500['sp500_log_return'] = np.log(sp500['sp500_close'] / sp500['sp500_close'].shift(1))
sp500['sp500_prev_log_return_1'] = sp500['sp500_log_return'].shift(1)
sp500['sp500_prev_log_return_2'] = sp500['sp500_log_return'].shift(2)
sp500['sp500_prev_log_return_3'] = sp500['sp500_log_return'].shift(3)
sp500['sp500_rolling_mean_return_7'] = sp500['sp500_log_return'].rolling(window=7).mean()
sp500['sp500_log_return_volatility'] = sp500['sp500_log_return'].rolling(window=7).std()

sp500.dropna(inplace=True)

sp500.to_csv('Preprocessed/sp500.csv', index=False)

In [25]:
bonds = pd.read_csv(bond_data)

bonds['Date'] = pd.to_datetime(bonds['Date'])
bonds = bonds.drop(columns='Volume')
bonds = rename_cols(bonds, ['Open', 'Close', 'High', 'Low'], ['bonds_open', 'bonds_close', 'bonds_high', 'bonds_low'])
bonds = fill_empty_dates(bonds, 'Date')

bonds.to_csv('Preprocessed/bonds.csv', index=False)

In [26]:
import pandas as pd

data = pd.read_csv('MacroData/2019_2024_inflation.csv')
month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
dates = pd.date_range('2019-01-01', '2024-12-31')
daily_data = []

for year in range(2019, 2025):
    for month, days_in_month in zip(months, month_days):
        inflation_value = data.loc[data['Year'] == year, month].values[0]
        month_dates = pd.date_range(f'{year}-{months.index(month)+1:02d}-01', periods=days_in_month)
        daily_data.extend([(date, inflation_value) for date in month_dates])

df_daily_inflation = pd.DataFrame(daily_data, columns=['Date', 'Inflation'])
df_daily_inflation.to_csv('Preprocessed/inflation_daily.csv', index=False)

-------------------------- BITCOIN DATA

In [27]:
btcprice = pd.read_csv(btc_prices)

btcprice['Date'] = pd.to_datetime(btcprice['Date'])
btcprice = rename_cols(btcprice, ['Open', 'Close', 'High', 'Low', 'Volume'], ['btc_open', 'btc_close', 'btc_high', 'btc_low', 'btc_volume'])
btcprice = fill_empty_dates(btcprice, 'Date')

# Add the btc_return column (percentage change of 'btc_close')
btcprice['btc_return'] = btcprice['btc_close'].pct_change()
btcprice['btc_log_return'] = np.log(btcprice['btc_close'] / btcprice['btc_close'].shift(1))
btcprice['btc_prev_log_return_1'] = btcprice['btc_log_return'].shift(1)
btcprice['btc_prev_log_return_2'] = btcprice['btc_log_return'].shift(2)
btcprice['btc_prev_log_return_3'] = btcprice['btc_log_return'].shift(3)
btcprice['btc_rolling_mean_return_7'] = btcprice['btc_log_return'].rolling(window=7).mean()
btcprice['btc_log_return_volatility'] = btcprice['btc_log_return'].rolling(window=7).std()

btcprice['log_btc_volume'] = np.log(btcprice['btc_volume'] + 1)

btcprice.dropna(inplace=True)

btcprice.to_csv('Preprocessed//btc_price.csv', index=False)


In [28]:
hashrate = pd.read_csv(hashrate_data)

hashrate = hashrate.drop(columns='x')
hashrate['datetime'] = pd.to_datetime(hashrate['datetime'])
hashrate = rename_cols(hashrate, ['datetime', 'y'], ['Date', 'hashrate'])
hashrate = fill_empty_dates(hashrate, 'Date')

hashrate.to_csv('Preprocessed/btc_hashrate.csv', index=False)

In [29]:
difficulty = pd.read_csv(difficulty_data)

difficulty = difficulty.drop(columns='x')
difficulty['datetime'] = pd.to_datetime(difficulty['datetime'])
difficulty = rename_cols(difficulty, ['datetime', 'y'], ['Date', 'difficulty'])
difficulty = fill_empty_dates(difficulty, 'Date')

difficulty.to_csv('Preprocessed/btc_difficulty.csv', index=False)

In [30]:
# Merge all datasets

btc_price = pd.read_csv('Preprocessed/btc_price.csv')
btc_hashrate = pd.read_csv('Preprocessed/btc_hashrate.csv')
btc_difficulty = pd.read_csv('Preprocessed/btc_difficulty.csv')

macro_sp500 = pd.read_csv('Preprocessed/sp500.csv')
macro_bonds = pd.read_csv('Preprocessed/bonds.csv')
macro_inflation = pd.read_csv('Preprocessed/inflation_daily.csv')

merged_data = btc_price
merged_data = merged_data.merge(btc_hashrate, on='Date', how='outer')
merged_data = merged_data.merge(btc_difficulty, on='Date', how='outer')
merged_data = merged_data.merge(macro_sp500, on='Date', how='outer')
merged_data = merged_data.merge(macro_bonds, on='Date', how='outer')
merged_data = merged_data.merge(macro_inflation, on='Date', how='outer')

merged_data = merged_data.dropna()

merged_data.to_csv('Preprocessed/data.csv', index=False)